In [2]:
import keras
from keras.layers import LSTM
import numpy as np

Using TensorFlow backend.


In [3]:
import data_loader as dl

In [4]:
data = dl.DataLoader()

In [5]:
X, Y, m = data.load()

In [6]:
from keras.preprocessing.sequence import pad_sequences

In [7]:
X_pad = pad_sequences(X, maxlen=m, padding='post')
Y_pad = pad_sequences(Y, maxlen=m, padding='post')

In [8]:
sample_weights = np.ones((273, m))
for i in xrange(273):
    for j in xrange(m):
        if (X_pad[i][j] == np.zeros(12)).all():
            sample_weights[i][j] = 0

In [11]:
from seq2seq.models import SimpleSeq2Seq

model = SimpleSeq2Seq(input_dim=12, hidden_dim=10, output_length=m, output_dim=12)

In [12]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              sample_weight_mode='temporal')

In [13]:
X_train, X_test = X_pad[:136, :], X_pad[136:, :]

In [14]:
Y_train, Y_test = Y_pad[:136, :], Y_pad[136:, :]

In [15]:
sample_weights_train, sample_weights_test = sample_weights[:136, :], sample_weights[136:, :]

In [16]:
# # for custom metrics
import numpy as K

# def weighted_accuracy(y_true, y_pred):
#     score_array *= sample_weights
# score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))

def weighted_accuracy(y_true, y_pred):
    # Only for testing
#     score_array = K.equal(K.argmax(y_true, axis=-1),
#                           K.argmax(y_pred, axis=-1))
#     score_array *= weights
#     score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))
#     return K.mean(score_array)
    total = 0.0
    count = 0.0
    for i, y_i in enumerate(y_true):
        for j, y_ij in enumerate(y_i):
            if sum(y_ij) > 0:
                total += y_ij[y_pred[i][j]]
                count += 1
    return total / count
    

In [18]:
hist = model.fit(X_train, Y_train, batch_size=136, nb_epoch=50, sample_weight=sample_weights_train)

Epoch 1/50
136/136 [==============================] - 19s - loss: 1.1921e-07
Epoch 2/50
136/136 [==============================] - 19s - loss: 1.1921e-07
Epoch 3/50
136/136 [==============================] - 19s - loss: 1.1921e-07
Epoch 4/50
136/136 [==============================] - 19s - loss: 1.1921e-07
Epoch 5/50
136/136 [==============================] - 19s - loss: 1.1921e-07
Epoch 6/50
136/136 [==============================] - 23s - loss: 1.1921e-07
Epoch 7/50
136/136 [==============================] - 21s - loss: 1.1921e-07
Epoch 8/50
136/136 [==============================] - 19s - loss: 1.1921e-07
Epoch 9/50
136/136 [==============================] - 19s - loss: 1.1921e-07
Epoch 10/50
136/136 [==============================] - 20s - loss: 1.1921e-07
Epoch 11/50
136/136 [==============================] - 18s - loss: 1.1921e-07
Epoch 12/50
136/136 [==============================] - 19s - loss: 1.1921e-07
Epoch 13/50
136/136 [==============================] - 22s - loss: 1.1921

In [19]:
Y_prediction = model.predict_classes(X_test, batch_size=5)

137/137 [==============================] - 100s   


In [21]:
# Y_prediction = to_categorical(Y_prediction)
# weighted_accuracy(Y_test, Y_prediction, sample_weights_test)
weighted_accuracy(Y_test, Y_prediction)

0.413776216690871

In [22]:
model.evaluate(X_test, Y_test, batch_size=137, sample_weight=sample_weights_test)

137/137 [==============================] - 2s


1.1922705311917525e-07

In [ ]:
Y_prediction[0]

In [ ]:
Y_prediction.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
len(model.get_weights())

In [ ]:
for w in model.get_weights():
    if len(w.shape) > 1:
        plt.figure()
        plt.imshow(w)

In [23]:
count = 0
for y in Y_prediction:
    if sum(y) > 0:
        count += 1
        print sum(y)
print count

0


In [25]:
sum(sum(Y_prediction))

0